In [0]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime, timedelta, date
from pyspark.sql.window import Window
import time
import os
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time
import random
import pandas as pd


In [0]:
%sql
SELECT * FROM test_new_patronage WHERE ICN IN (1035452953,1018105031, 1003457903)

In [0]:
df = spark.read.csv("dbfs:/mnt/ci-vadir-shared/CPIDODIEX_20250906_spool.csv", header=True)
df.filter(col("PTCPNT_ID") == 2237252 ).display()

In [0]:
%sql
SELECT * FROM delta.`/mnt/Patronage/SCD_Staging` WHERE ICN IN (1035452953,1018105031, 1003457903)

In [0]:
%sql
SELECT * FROM delta.`/mnt/ci-patronage/delta_tables/correlation_lookup` WHERE MVIPersonICN =1035452953

In [0]:
%sql
SELECT * FROM
test_new_patronage
--  delta.`/mnt/Patronage/Caregivers_Staging_New` 
WHERE SDP_Event_Created_Timestamp = '2025-09-11T16:23:49.000+00:00'
--  WHERE edipi IS NULL AND Batch_CD = 'CG'

In [0]:
%sql
DESCRIBE EXTENDED test_new_patronage

In [0]:
%sql
-- SELECT * FROM delta.`/mnt/Patronage/Caregivers_Staging_New` ORDER BY SDP_Event_Created_Timestamp DESC
SELECT * FROM test_new_patronage ORDER BY SDP_Event_Created_Timestamp DESC


In [0]:
%sql
        SELECT 
            count(*), "Staging_CG_Counts" AS StagingCaregivers
        FROM delta.`/mnt/Patronage/Caregivers_Staging_New`
        WHERE SDP_Event_Created_Timestamp >= '2025-09-08 20:00:00'
        AND SDP_Event_Created_Timestamp <= DATE('2025-09-12 00:00:00')
        AND EDIPI IS NOT NULL
        AND Applicant_Type = 'Primary Caregiver'
        AND (Status_Termination_Date IS NULL 
             OR Status_Termination_Date >= curdate()
             OR Status IN ('Approved', 'Pending Revocation/Discharge'))
UNION ALL
        SELECT 
            count(*), "Patronage_CG_Counts" AS StagingCaregivers
        FROM test_new_patronage
        WHERE SDP_Event_Created_Timestamp >= '2025-09-08 20:00:00'
        AND SDP_Event_Created_Timestamp <= DATE('2025-09-12 00:00:00')
        AND EDIPI IS NOT NULL
        AND Applicant_Type = 'Primary Caregiver'
        AND (Status_Termination_Date IS NULL 
             OR Status_Termination_Date >= to_char(curdate(), 'yyyyMMdd')
             OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))
        AND RecordStatus IS TRUE
        AND Batch_CD = 'CG'
UNION ALL
SELECT 
            count(*), "StagingSCDCount" as StagingCount
        FROM delta.`/mnt/Patronage/SCD_Staging`
        WHERE SDP_Event_Created_Timestamp >= DATE('2025-09-09')
        AND SDP_Event_Created_Timestamp <= DATE('2025-09-12 00:00:00')
        AND edipi IS NOT NULL
UNION ALL
SELECT 
            count(*), "PatronageSCDCount" as PatronageCount
        FROM test_new_patronage
        WHERE SDP_Event_Created_Timestamp >= DATE('2025-09-09')
        AND SDP_Event_Created_Timestamp <= DATE('2025-09-12 00:00:00')
        AND edipi IS NOT NULL
        AND RecordStatus IS TRUE
        AND Batch_CD = 'SCD'


In [0]:
%sql
        SELECT 
            ICN, "Staging_CG_Counts" AS StagingCaregivers
        FROM delta.`/mnt/Patronage/Caregivers_Staging_New`
        WHERE SDP_Event_Created_Timestamp >= '2025-09-08 20:00:00'
        AND SDP_Event_Created_Timestamp <= DATE('2025-09-12 00:00:00')
        AND EDIPI IS NOT NULL
        AND Applicant_Type = 'Primary Caregiver'
        AND (Status_Termination_Date IS NULL 
             OR Status_Termination_Date >= curdate()
             OR Status IN ('Approved', 'Pending Revocation/Discharge'))
UNION ALL
        SELECT 
            ICN, "Patronage_CG_Counts" AS StagingCaregivers
        FROM test_new_patronage
        WHERE SDP_Event_Created_Timestamp >= '2025-09-08 20:00:00'
        AND SDP_Event_Created_Timestamp <= DATE('2025-09-12 00:00:00')
        AND EDIPI IS NOT NULL
        AND Applicant_Type = 'Primary Caregiver'
        AND (Status_Termination_Date IS NULL 
             OR Status_Termination_Date >= to_char(curdate(), 'yyyyMMdd')
             OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))
        AND RecordStatus IS TRUE
        AND Batch_CD = 'CG'

In [0]:

%sql
        SELECT 
           *
        FROM delta.`/mnt/Patronage/Caregivers_Staging_New`
        WHERE SDP_Event_Created_Timestamp >= '2025-09-08 20:00:00'
        AND SDP_Event_Created_Timestamp <= DATE('2025-09-12 00:00:00')
        AND EDIPI IS NOT NULL
        AND Applicant_Type = 'Primary Caregiver'
        AND (Status_Termination_Date IS NULL 
             OR Status_Termination_Date >= curdate()
             OR Status IN ('Approved', 'Pending Revocation/Discharge'))
        AND ICN in (1020275775, 1077174952, 1077779013, 1083037601)

In [0]:
%sql
SELECT * FROM test_new_patronage WHERE ICN IN (1020275775, 1077174952, 1077779013, 1083037601)

In [0]:
%sql
SELECT 
            *
        -- FROM delta.`/mnt/Patronage/Caregivers_Staging_New`
        FROM test_new_patronage
        WHERE SDP_Event_Created_Timestamp >= '2025-09-09 20:00:00'
        -- AND SDP_Event_Created_Timestamp <= DATE('2025-09-09')
        AND EDIPI IS NOT NULL
        AND Applicant_Type = 'Primary Caregiver'
        AND (Status_Termination_Date IS NULL 
             OR Status_Termination_Date >= curdate()
             OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))

In [0]:
 %sql
 SELECT 
           *
        -- FROM test_new_patronage
         FROM delta.`/mnt/Patronage/Caregivers_Staging_New`
        WHERE SDP_Event_Created_Timestamp >= (SELECT max(DATEADD(hour, -4, checkpoint_timestamp)) as utctime FROM dmdc_checkpoint )
        AND SDP_Event_Created_Timestamp <= DATEADD(hour, -4, ('2025-09-08'))
        AND EDIPI IS NOT NULL
        AND Applicant_Type = 'Primary Caregiver'
        AND (Status_Termination_Date IS NULL 
             OR Status_Termination_Date >= curdate()
             OR Status IN ('Approved', 'Pending Revocation/Discharge'))
        AND edipi IN (
1116972995,
1138557436,
1185542277,
1278416611,
1293389000,
1407557294,
1604478657,
1610358973,
1612998031,
1618867427,
1633156359,
1634182116,
1636984457,
1640764291,
1642242055,
1642834840,
1644535947,
1644621134,
1645053601,
1646273605
 ) 


In [0]:
1133163073CG    20250903                  
90   1628661037CG    20250905                  
91   1609553304CG    20250606          20251112
92   1629008787CG    20240215          20251013
93   1636408260CG    20241223          20251130
94   1182462647CG    20250905                  
95   1645739899CG    20250905                  
96   1409700671CG    20250902                  
97   1407557294CG    20250828                  
98   1412245624CG    20160718          20251003
99   1642490881CG    20250429          20251203
100  1644618117CG    20250827                  
101  1238825764CG    20120608          20251004
102  1410783610CG    20250528          20251201
103  1644047926CG    20250902                  
104  1603987906CG    20250905                  
105  1613057788CG    20240618          20251112
106  1097634943CG    20240904          20251121

In [0]:
%sql
SELECT  *   FROM delta.`/mnt/Patronage/Caregivers_Staging_New` WHERE 
         
 edipi IN (
1116972995,
1138557436,
1185542277,
1278416611,
1293389000,
1407557294,
1604478657,
1610358973,
1612998031,
1618867427,
1633156359,
1634182116,
1636984457,
1640764291,
1642242055,
1642834840,
1644535947,
1644621134,
1645053601,
1646273605
 )


In [0]:
display(dbutils.fs.ls('/mnt'))

In [0]:
display(dbutils.fs.ls("/mnt/ci-mvi/"))


In [0]:
edipi_lookup = (
                spark.read.format("delta")
                .load("/mnt/Patronage/identity_correlations"))

In [0]:
 %sql
 SELECT 
           *
        -- FROM test_new_patronage
         FROM delta.`/mnt/Patronage/Caregivers_Staging_New`
        WHERE SDP_Event_Created_Timestamp >= (SELECT max(DATEADD(hour, -4, checkpoint_timestamp)) as utctime FROM dmdc_checkpoint )
        AND SDP_Event_Created_Timestamp <= DATEADD(hour, -4, ('2025-09-08'))
        AND EDIPI IS NOT NULL
        AND Applicant_Type = 'Primary Caregiver'
        AND (Status_Termination_Date IS NULL 
             OR Status_Termination_Date >= date_add(day, -4, curdate())
             OR Status IN ('Approved', 'Pending Revocation/Discharge'))
        AND edipi IN (
1116972995,
1138557436,
1185542277,
1278416611,
1293389000,
1407557294,
1604478657,
1610358973,
1612998031,
1618867427,
1633156359,
1634182116,
1636984457,
1640764291,
1642242055,
1642834840,
1644535947,
1644621134,
1645053601,
1646273605
 ) 


In [0]:
%sql
SELECT max(DATEADD(hour, -4, checkpoint_timestamp)) as utctime FROM dmdc_checkpoint 

In [0]:
 %sql
 SELECT 
           *
        -- FROM test_new_patronage
         FROM delta.`/mnt/Patronage/Caregivers_Staging_New`
         WHERE         
        SDP_Event_Created_Timestamp >= DATE('2025-09-05')
        and SDP_Event_Created_Timestamp <= DATE('2025-09-08')
        and EDIPI is not null
        and Applicant_Type = 'Primary Caregiver'
        and (Status_Termination_Date is NULL OR Status_Termination_Date >= curdate()
            OR Status IN ('Approved', 'Pending Revocation/Discharge'))
        and EDIPI is not null
        AND edipi IN (
1116972995,
1138557436,
1185542277,
1278416611,
1293389000,
1407557294,
1604478657,
1610358973,
1612998031,
1618867427,
1633156359,
1634182116,
1636984457,
1640764291,
1642242055,
1642834840,
1644535947,
1644621134,
1645053601,
1646273605
 ) 

In [0]:
def compare_correlation_tables_simplified():
    """
    A simplified approach to compare correlation tables directly
    """
    print("=" * 60)
    print("CORRELATION TABLES COMPARISON REPORT")
    print("=" * 60)
    
    # Load both tables
    test_table = spark.sql("SELECT * FROM delta.`/mnt/Patronage/identity_correlations`")
    prod_table = spark.table("correlation_lookup")
    
    # Get counts
    test_count = test_table.count()
    prod_count = prod_table.count()
    
    print(f"Record counts:")
    print(f"   correlation_lookup_test: {test_count:,}")
    print(f"   correlation_lookup: {prod_count:,}")
    
    # Join tables to find matches and differences
    joined_df = test_table.alias("test").join(
        prod_table.alias("prod"),
        test_table.MVIPersonICN == prod_table.MVIPersonICN,
        "outer"
    )
    
    # Count records in each category
    only_in_test = joined_df.filter(col("prod.MVIPersonICN").isNull()).count()
    only_in_prod = joined_df.filter(col("test.MVIPersonICN").isNull()).count()
    common_records = joined_df.filter(
        col("test.MVIPersonICN").isNotNull() & 
        col("prod.MVIPersonICN").isNotNull()
    ).count()
    
    print(f"\nCoverage analysis:")
    print(f"   Records in both tables: {common_records:,}")
    print(f"   Only in test table: {only_in_test:,}")
    print(f"   Only in prod table: {only_in_prod:,}")
    
    # Column-by-column comparison for common records
    common_df = joined_df.filter(
        col("test.MVIPersonICN").isNotNull() & 
        col("prod.MVIPersonICN").isNotNull()
    )
    
    # Define columns to compare
    columns_to_check = ["participant_id", "edipi", "va_profile_id", "ICNStatus", "Last_Modified_Date"]
    
    print(f"\nColumn-by-column comparison:")
    
    for column in columns_to_check:
        mismatch_condition = (
            coalesce(col(f"test.{column}"), lit("")) != 
            coalesce(col(f"prod.{column}"), lit(""))
        )
        
        mismatch_count = common_df.filter(mismatch_condition).count()
        match_rate = ((common_records - mismatch_count) / common_records * 100) if common_records > 0 else 0
        
        print(f"\n{column}:")
        print(f"   Matching: {common_records - mismatch_count:,}")
        print(f"   Mismatched: {mismatch_count:,}")
        print(f"   Match rate: {match_rate:.2f}%")
        
        if mismatch_count > 0:
            print(f"\nSample mismatches for {column}:")
            common_df.filter(mismatch_condition).select(
                col("test.MVIPersonICN"), 
                col(f"test.{column}").alias(f"test_{column}"),
                col(f"prod.{column}").alias(f"prod_{column}")
            ).show(10, truncate=False)
    
    # Overall summary
    any_mismatch_condition = functools.reduce(
        lambda a, b: a | b, 
        [
            coalesce(col(f"test.{c}"), lit("")) != coalesce(col(f"prod.{c}"), lit(""))
            for c in columns_to_check
        ]
    )
    
    any_mismatch_count = common_df.filter(any_mismatch_condition).count()
    overall_match_rate = ((common_records - any_mismatch_count) / common_records * 100) if common_records > 0 else 0
    
    print(f"\nOverall Results:")
    print(f"   Perfect matches (all columns): {common_records - any_mismatch_count:,}")
    print(f"   Records with any mismatch: {any_mismatch_count:,}")
    print(f"   Overall match rate: {overall_match_rate:.2f}%")

In [0]:
compare_correlation_tables_simplified()

In [0]:
%sql
SELECT * FROM test_new_patronage WHERE ICN IS NULL and Batch_CD = 'SCD'

In [0]:
%sql

WITH sink_scd AS (
  SELECT 
    ICN,
    edipi,
    participant_id
  FROM test_new_patronage 
  WHERE Batch_CD = 'SCD' AND RecordStatus = True
),

josh_scd AS (
  SELECT 
    ICN,
    edipi,
    participant_id
  FROM delta.`/mnt/Patronage/SCD_Staging`
  -- WHERE Batch_CD = 'SCD'
),

presence_comparison AS (
  SELECT 
    COALESCE(s.ICN, j.ICN) as ICN,
    
    -- Record existence flags
    CASE WHEN s.ICN IS NOT NULL THEN 'YES' ELSE 'NO' END as in_sink_table,
    CASE WHEN j.ICN IS NOT NULL THEN 'YES' ELSE 'NO' END as in_josh_table,
    
    -- Values for comparison
    s.edipi as sink_edipi,
    j.edipi as josh_edipi,
    s.participant_id as sink_participant_id,
    j.participant_id as josh_participant_id
    
  FROM sink_scd s
  FULL OUTER JOIN josh_scd j ON s.ICN = j.ICN
),

mismatch_summary AS (
  SELECT 
    'SUMMARY' as record_type,
    '' as ICN,
    '' as mismatch_details,
    CAST(COUNT(*) AS STRING) as total_unique_icns,
    CAST(SUM(CASE WHEN in_sink_table = 'YES' THEN 1 ELSE 0 END) AS STRING) as records_in_sink,
    CAST(SUM(CASE WHEN in_josh_table = 'YES' THEN 1 ELSE 0 END) AS STRING) as records_in_josh,
    CAST(SUM(CASE WHEN in_sink_table = 'YES' AND in_josh_table = 'NO' THEN 1 ELSE 0 END) AS STRING) as only_in_sink,
    CAST(SUM(CASE WHEN in_sink_table = 'NO' AND in_josh_table = 'YES' THEN 1 ELSE 0 END) AS STRING) as only_in_josh,
    '' as sink_edipi,
    '' as josh_edipi,
    '' as sink_participant_id,
    '' as josh_participant_id
  FROM presence_comparison
),

detailed_mismatches AS (
  SELECT 
    'MISMATCH_DETAIL' as record_type,
    ICN,
    CASE 
      WHEN in_sink_table = 'YES' AND in_josh_table = 'NO' THEN 'ONLY_IN_SINK_TABLE'
      WHEN in_sink_table = 'NO' AND in_josh_table = 'YES' THEN 'ONLY_IN_JOSH_TABLE'
      ELSE 'UNKNOWN_MISMATCH'
    END as mismatch_details,
    '' as total_unique_icns,
    in_sink_table as records_in_sink,
    in_josh_table as records_in_josh,
    '' as only_in_sink,
    '' as only_in_josh,
    COALESCE(sink_edipi, 'NULL') as sink_edipi,
    COALESCE(josh_edipi, 'NULL') as josh_edipi,
    COALESCE(sink_participant_id, 'NULL') as sink_participant_id,
    COALESCE(josh_participant_id, 'NULL') as josh_participant_id
  FROM presence_comparison
  WHERE in_sink_table != in_josh_table  -- Only show records that exist in one table but not the other
)

-- Combine summary and details
SELECT * FROM mismatch_summary
UNION ALL
SELECT * FROM detailed_mismatches
ORDER BY 
  CASE WHEN record_type = 'SUMMARY' THEN 0 ELSE 1 END,
  ICN
LIMIT 101;  -- 1 summary row + 100 detail rows

In [0]:
# Step 1: Check if Josh's table has data
josh_total = spark.sql("""
    SELECT COUNT(*) as total_records, COUNT(DISTINCT ICN) as unique_icns
    FROM delta.`/mnt/Patronage/SCD_Staging`
    WHERE Batch_CD = 'SCD'
""").collect()[0]

print(f"Josh's SCD table: {josh_total['total_records']:,} total records, {josh_total['unique_icns']:,} unique ICNs")

# Step 2: Check your sink table
sink_total = spark.sql("""
    SELECT COUNT(*) as total_records, COUNT(DISTINCT ICN) as unique_icns
    FROM test_new_patronage
    WHERE Batch_CD = 'SCD' AND RecordStatus = True
""").collect()[0]

print(f"Your sink table: {sink_total['total_records']:,} total records, {sink_total['unique_icns']:,} unique ICNs")

# Step 3: Check overlap between tables
overlap = spark.sql("""
    WITH josh_icns AS (
        SELECT DISTINCT ICN
        FROM delta.`/mnt/Patronage/SCD_Staging`
        WHERE Batch_CD = 'SCD'
    ),
    sink_icns AS (
        SELECT DISTINCT ICN
        FROM test_new_patronage
        WHERE Batch_CD = 'SCD' AND RecordStatus = True
    )
    SELECT 
        (SELECT COUNT(*) FROM josh_icns) as josh_unique_icns,
        (SELECT COUNT(*) FROM sink_icns) as sink_unique_icns,
        COUNT(j.ICN) as icns_in_both,
        (SELECT COUNT(*) FROM josh_icns) - COUNT(j.ICN) as josh_only,
        (SELECT COUNT(*) FROM sink_icns) - COUNT(j.ICN) as sink_only
    FROM josh_icns j
    INNER JOIN sink_icns s ON j.ICN = s.ICN
""").collect()[0]

print(f"\nOverlap Analysis:")
print(f"  Josh unique ICNs: {overlap['josh_unique_icns']:,}")
print(f"  Sink unique ICNs: {overlap['sink_unique_icns']:,}")
print(f"  ICNs in both tables: {overlap['icns_in_both']:,}")
print(f"  ICNs only in Josh's table: {overlap['josh_only']:,}")
print(f"  ICNs only in sink table: {overlap['sink_only']:,}")

# Step 4: Sample some ICNs that should be missing
missing_samples = spark.sql("""
    SELECT j.ICN, j.edipi, j.participant_id, j.SDP_Event_Created_Timestamp
    FROM delta.`/mnt/Patronage/SCD_Staging` j
    LEFT JOIN (
        SELECT DISTINCT ICN 
        FROM test_new_patronage 
        WHERE Batch_CD = 'SCD' AND RecordStatus = True
    ) s ON j.ICN = s.ICN
    WHERE j.Batch_CD = 'SCD' AND s.ICN IS NULL
    LIMIT 5
""")

print(f"\nSample missing records:")
missing_samples.display()

In [0]:
overlap_df.display()

In [0]:
def integrate_historical_scd_data(
    historical_table_path="/mnt/Patronage/SCD_Staging",
    sink_table_name="test_new_patronage",
    execute_integration=False
):
    """
    Integrate historical SCD data from staging table into patronage pipeline.
    
    This function performs two key operations:
    1. Insert new historical veterans that don't exist in the sink table
    2. Update existing veterans with earlier Status_Begin_Date from historical data
    
    Args:
        historical_table_path (str): Path to historical SCD staging Delta table
        sink_table_name (str): Target patronage table name
        execute_integration (bool): If True, executes actual data modifications
        
    Returns:
        dict: Integration summary with record counts and actions taken
    """
    from delta.tables import DeltaTable
    
    print("🚀 Starting Historical SCD Data Integration...")
    print("="*70)
    
    try:
        # Phase 1: Insert New Historical Records
        print("📥 PHASE 1: INSERT NEW HISTORICAL RECORDS")
        print("="*70)
        
        historical_insert_df = spark.sql(f"""
            SELECT 
                CAST(h.edipi AS STRING) as edipi,
                CAST(h.ICN AS STRING) as ICN,
                CAST(NULL AS STRING) as Veteran_ICN,
                CAST(h.participant_id AS STRING) as participant_id,
                CAST(h.Batch_CD AS STRING) as Batch_CD,
                CAST(NULL AS STRING) as Applicant_Type,
                CAST(NULL AS STRING) as Caregiver_Status,
                CAST(h.SC_Combined_Disability_Percentage AS STRING) as SC_Combined_Disability_Percentage,
                CAST(h.PT_Indicator AS STRING) as PT_Indicator,
                CAST(h.Individual_Unemployability AS STRING) as Individual_Unemployability,
                CAST(h.Status_Begin_Date AS STRING) as Status_Begin_Date,
                CAST(h.Status_Last_Update AS STRING) as Status_Last_Update,
                CAST(h.Status_Termination_Date AS STRING) as Status_Termination_Date,
                CAST(h.SDP_Event_Created_Timestamp AS TIMESTAMP) as SDP_Event_Created_Timestamp,
                
                -- Generate filename using historical timestamp
                CAST(CONCAT(
                    'dbfs:/mnt/ci-vadir-shared/CPIDODIEX_',
                    DATE_FORMAT(h.SDP_Event_Created_Timestamp, 'yyyyMMdd'),
                    '_spool.csv'
                ) AS STRING) as filename,
                
                -- RecordLastUpdated = NULL for new records
                CAST(NULL AS DATE) as RecordLastUpdated,
                
                -- RecordStatus = TRUE (active historical records)
                CAST(TRUE AS BOOLEAN) as RecordStatus,
                CAST(FALSE AS BOOLEAN) as sentToDoD,
                
                -- Pipeline-aligned change_log format
                CAST(CONCAT(
                    'SC_Combined_Disability_Percentage old value: Null changed to new value: ',
                    h.SC_Combined_Disability_Percentage
                ) AS STRING) as change_log,
                
                -- Pipeline-aligned RecordChangeStatus
                CAST('New Record' AS STRING) as RecordChangeStatus
                
            FROM delta.`{historical_table_path}` h
            WHERE h.Batch_CD = 'SCD'
                AND h.ICN NOT IN (
                    SELECT DISTINCT ICN 
                    FROM {sink_table_name} 
                    WHERE Batch_CD = 'SCD' 
                        AND RecordStatus = True
                        AND ICN IS NOT NULL
                )
                -- AND h.SDP_Event_Created_Timestamp <= '2025-08-23T00:00:00.000+00:00'
        """)
        
        phase1_count = historical_insert_df.count()
        print(f"📊 Phase 1 - New historical records to insert: {phase1_count:,}")
        
        # Display sample records for verification
        print("\n📋 Sample Phase 1 records:")
        historical_insert_df.select(
            "ICN", "edipi", "participant_id", "SC_Combined_Disability_Percentage", 
            "PT_Indicator", "Status_Begin_Date", "RecordStatus", "change_log", "filename"
        ).display()
        
        # Phase 2: Update Existing Records with Earlier Status_Begin_Date
        print("\n" + "="*70)
        print("🔄 PHASE 2: UPDATE STATUS_BEGIN_DATE WITH EARLIEST DATES")
        print("="*70)
        
        # Prepare update source data
        status_date_update_source = spark.sql(f"""
            SELECT 
                h.ICN as update_icn,
                h.Status_Begin_Date as historical_status_begin_date,
                s.Status_Begin_Date as current_status_begin_date
                
            FROM delta.`{historical_table_path}` h
            INNER JOIN {sink_table_name} s ON h.ICN = s.ICN
            WHERE h.Batch_CD = 'SCD'
                AND s.Batch_CD = 'SCD' 
                AND s.RecordStatus = True
                AND h.Status_Begin_Date < s.Status_Begin_Date  -- Only when historical has earlier date
                -- AND h.SDP_Event_Created_Timestamp <= '2025-08-23T00:00:00.000+00:00'
        """)
        
        phase2_count = status_date_update_source.count()
        print(f"📊 Phase 2 - Records where historical data has earlier Status_Begin_Date: {phase2_count:,}")
        
        print("\n📋 Sample Phase 2 updates:")
        status_date_update_source.display()
        
        # Execute integration if requested
        if execute_integration:
            print(f"\n🔄 EXECUTING INTEGRATION...")
            print(f"🔄 Executing Phase 1 insertion of {phase1_count:,} records...")
            
            # Execute Phase 1: Insert new historical records
            historical_insert_df.write \
                .format("delta") \
                .mode("append") \
                .option("mergeSchema", "true") \
                .saveAsTable(sink_table_name)
            
            print(f"✅ Phase 1 completed - {phase1_count:,} new historical records inserted")
            
            # Execute Phase 2: Update Status_Begin_Date for existing records
            if phase2_count > 0:
                print(f"🔄 Executing Phase 2 Status_Begin_Date updates for {phase2_count:,} records...")
                
                targetTable = DeltaTable.forPath(spark, f"dbfs:/user/hive/warehouse/{sink_table_name}")
                
                targetTable.alias("target").merge(
                    status_date_update_source.alias("source"),
                    "target.ICN = source.update_icn AND target.Batch_CD = 'SCD' AND target.RecordStatus = True"
                ).whenMatchedUpdate(
                    set={
                        "Status_Begin_Date": "source.historical_status_begin_date"  # Update to historical earlier date
                    }
                ).execute()
                
                print(f"✅ Phase 2 completed - Status_Begin_Date updated to earliest dates for {phase2_count:,} records")
            else:
                print("ℹ️  Phase 2 skipped - no records need Status_Begin_Date updates")
        else:
            print(f"\n🔍 INTEGRATION PREVIEW MODE")
            print(f"📝 To execute integration, call function with execute_integration=True")
        
        # Integration Summary
        print("\n" + "="*70)
        print("📊 INTEGRATION SUMMARY & VERIFICATION")
        print("="*70)
        
        summary = {
            'phase1_new_records': phase1_count,
            'phase2_status_updates': phase2_count,
            'total_impact': phase1_count + phase2_count,
            'executed': execute_integration
        }
        
        print(f"""
🎯 HISTORICAL SCD DATA INTEGRATION SUMMARY:

📥 Phase 1 - New Historical Records:
   • Records to insert: {summary['phase1_new_records']:,}
   • Action: Insert completely new veterans from historical data

🔄 Phase 2 - Status_Begin_Date Updates:
   • Records to update: {summary['phase2_status_updates']:,}
   • Logic: Update ONLY when historical data has earlier Status_Begin_Date
   • Action: Preserve earliest disability start dates

🏆 TOTAL IMPACT:
   • New veterans added: {summary['phase1_new_records']:,}
   • Existing veterans with corrected start dates: {summary['phase2_status_updates']:,}
   • Complete historical coverage: Enhanced from historical staging data
   • Execution status: {'✅ COMPLETED' if summary['executed'] else '📋 PREVIEW ONLY'}
        """)
        
        if not execute_integration:
            print(f"\n🚀 Ready for execution! Call with execute_integration=True to proceed.")
        
        print("="*70)
        
        return summary
        
    except Exception as e:
        print(f"❌ CRITICAL ERROR during historical data integration:")
        print(f"❌ Error details: {str(e)}")
        print(f"❌ Integration failed - please review configuration and try again")
        
        return {
            'phase1_new_records': 0,
            'phase2_status_updates': 0,
            'total_impact': 0,
            'executed': False,
            'error': str(e)
        }


# Simplified execution function
def execute_historical_integration():
    """
    Execute historical SCD data integration with default parameters.
    This is the main entry point for historical data backfill.
    """
    print("🎯 Executing Historical SCD Data Integration...")
    
    # Preview mode first
    preview_results = integrate_historical_scd_data(
        historical_table_path="/mnt/Patronage/SCD_Staging",
        sink_table_name="test_new_patronage",
        execute_integration=False  # Preview mode
    )
    
    print(f"\n📋 Integration Preview Results:")
    print(f"   New historical records: {preview_results['phase1_new_records']:,}")
    print(f"   Status_Begin_Date updates: {preview_results['phase2_status_updates']:,}")
    print(f"   Total impact: {preview_results['total_impact']:,}")
    
    return preview_results

In [0]:
# Preview all historical data integration
# preview = integrate_historical_scd_data()

# Execute all historical data integration
results = integrate_historical_scd_data(execute_integration=False)

# Simple execution using wrapper
# summary = execute_historical_integration()

In [0]:
print(results)

In [0]:
%sql
Select * from delta.`/mnt/Patronage/SCD_Staging` where ICN in
(
'1036250106',
'1042872004',
'1018270220',
'1050961503',
'1015378617',
'1005526019',
'1023883977',
'1034605364',
'1058203557',
'1034690212'
)

In [0]:
%sql
SELECT * FROM mypatronage_test_new WHERE participant_id = 30529860

In [0]:
josh_insert_df = spark.sql("""
    SELECT
        CAST(j.edipi AS STRING) as edipi,
        CAST(j.ICN AS STRING) as ICN,
        CAST(NULL AS STRING) as Veteran_ICN,
        CAST(j.participant_id AS STRING) as participant_id,
        CAST(j.Batch_CD AS STRING) as Batch_CD,
        CAST(NULL AS STRING) as Applicant_Type,
        CAST(NULL AS STRING) as Caregiver_Status,
        CAST(j.SC_Combined_Disability_Percentage AS STRING) as SC_Combined_Disability_Percentage,
        CAST(j.PT_Indicator AS STRING) as PT_Indicator,
        CAST(j.Individual_Unemployability AS STRING) as Individual_Unemployability,
        CAST(j.Status_Begin_Date AS STRING) as Status_Begin_Date,
        CAST(j.Status_Last_Update AS STRING) as Status_Last_Update,
        CAST(j.Status_Termination_Date AS STRING) as Status_Termination_Date,
        CAST(j.SDP_Event_Created_Timestamp AS TIMESTAMP) as SDP_Event_Created_Timestamp,
       
        CAST(CONCAT(
            'dbfs:/mnt/ci-vadir-shared/CPIDODIEX_',
            DATE_FORMAT(j.SDP_Event_Created_Timestamp, 'yyyyMMdd'),
            '_spool.csv'
        ) AS STRING) as filename,
       
        CAST(NULL AS DATE) as RecordLastUpdated,
        CAST(TRUE AS BOOLEAN) as RecordStatus,
        CAST(FALSE AS BOOLEAN) as sentToDoD,
       
        CAST(CONCAT(
            'SC_Combined_Disability_Percentage old value: Null changed to new value: ',
            j.SC_Combined_Disability_Percentage
        ) AS STRING) as change_log,
       
        CAST('New Record' AS STRING) as RecordChangeStatus
       
    FROM delta.`/mnt/Patronage/SCD_Staging` j
    WHERE j.Batch_CD = 'SCD'
        AND j.ICN NOT IN (
            SELECT DISTINCT ICN
            FROM mypatronage_test_new
            WHERE Batch_CD = 'SCD'
                AND RecordStatus = True
                AND ICN IS NOT NULL
        )
""")

In [0]:
josh_insert_df.display()

In [0]:
def add_numbers(a, b):
    return a + b

In [0]:
import unittest

class TestAddition(unittest.TestCase):

    def test_add_numbers(self):
        self.assertEqual(add_numbers(2, 3), 5, "Should be 5")
        self.assertEqual(add_numbers(-1, 1), 0, "Should be 0")
        self.assertEqual(add_numbers(0, 0), 0, "Should be 0")

if __name__ == '__main__':
    result = unittest.main(argv=['first-arg-is-ignored'], exit=False)
    if not result.result.wasSuccessful():
        print("Unit tests failed. Exiting the notebook.")
        # dbutils.notebook.exit("Unit tests failed. Exiting the notebook.")

In [0]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

In [0]:
import unittest


class TestMathUtils(unittest.TestCase):
    def add(num1, num2):
        return num1+num2
    def test_add_positive_numbers(self):
        self.assertEqual(add(2,2), 5)
if __name__ == '__main':
    unittest.main()

In [0]:
now = datetime.now()

# Create the first date of the current year
first_date = datetime(now.year, 1, 1)
print(first_date)

In [0]:
%sql
SELECT
  version,
  timestamp,
  operation,
  cast(operationMetrics.numTargetRowsInserted as INT) as TotalInsertedRecords,
  cast(operationMetrics.numTargetRowsUpdated as INT) as UpdatedRecords,
  (cast(operationMetrics.numTargetRowsInserted as INT) - cast(operationMetrics.numTargetRowsUpdated as INT)) as NewRecords
FROM (DESCRIBE HISTORY mypat_test)
WHERE operation NOT IN ("CREATE TABLE", "OPTIMIZE")
ORDER BY version DESC
-- LIMIT 10;

In [0]:
%sql
SELECT
  version,
  timestamp,
  operation,
  cast(operationMetrics.numTargetRowsInserted as INT) as TotalInsertedRecords,
  cast(operationMetrics.numTargetRowsUpdated as INT) as UpdatedRecords,
  (cast(operationMetrics.numTargetRowsInserted as INT) - cast(operationMetrics.numTargetRowsUpdated as INT)) as NewRecords
FROM (DESCRIBE HISTORY mypatronage_test_new)
WHERE operation NOT IN ("CREATE TABLE", "OPTIMIZE")
ORDER BY version DESC
-- LIMIT 10;

In [0]:
%sql
SELECT change_log, RecordChangeStatus, RecordStatus, RecordLastUpdated FROM mypatronage_test111
WHERE PT_Indicator = 'Y'

In [0]:
%sql
SELECT DISTINCT SDP_Event_Created_Timestamp, count(*)
FROM mypatronage_test111
-- WHERE Batch_CD = 'SCD'
GROUP BY ALL

In [0]:
%sql
SELECT BATCH_CD, count(*) FROM mypatronage_test111
GROUP BY ALL

In [0]:
%sql
SELECT BATCH_CD, count(*) FROM mypatronage_test_new
WHERE BATCH_CD = 'CG' AND RecordStatus IS TRUE and Applicant_Type = 'Primary Caregiver'
GROUP BY ALL

In [0]:
containerPath = '/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/'
mostRecentRequestPath = None
for entity in dbutils.fs.ls(containerPath):
  print(entity.name)
  print(entity)

In [0]:
df = spark.read.format('delta').load(containerPath + 'DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/')
df.createOrReplaceTempView('DOD_PATRONAGE_SCD_PT') # name of the Delta Table you are reading

In [0]:
df = spark.read.format("delta").load("/mnt/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/")

In [0]:
%sql
SELECT * FROM DELTA.`/mnt/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/`
-- `/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/`
                    

In [0]:
%sql
SELECT PTCPNT_VET_ID, count(1)
FROM delta.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT`
GROUP BY ALL
HAVING count(1) > 1

In [0]:
%sql
SELECT distinct PT_Indicator, count(*)  FROM 
mypatronage_test_new WHERE RecordStatus IS TRUE and Batch_CD = 'SCD'
-- delta.`/mnt/Patronage/SCD_Staging` 
GROUP BY ALL
-- PT_Indicator	count(1)
-- Y	2153366
-- N	4398097

In [0]:
%sql
SELECT distinct PT_Indicator, count(*)  FROM mypat_test WHERE RecordStatus IS TRUE AND Batch_CD = "SCD" GROUP BY ALL
-- 6386058


In [0]:
cg_unix_start_time = int(time.mktime(datetime(2024, 12, 18, 23, 59, 59).timetuple())) * 1000
print(cg_unix_start_time)
others_unix_start_time = int(time.mktime(datetime(2023, 11, 1, 0, 0, 0).timetuple())) * 1000
print(others_unix_start_time)
date_today = datetime.today().date()
print(date_today) #date only
date_timestamp = datetime.today()
print(date_timestamp) #timestamp
print(date_timestamp.strftime("%A, %d %B %Y [Month Number: %m]" )) #formatting
print(int(time.mktime(date_today.timetuple())))
# print(int(time.mktime(cg_unix_start_time.timetuple()))) #1698796800000



In [0]:
%sql
with DateSeries as (
SELECT EXPLODE(SEQUENCE( DATE('2025-01-01'), last_day('2025-12-01'), INTERVAL 1 DAY )) as Year_2025
)
SELECT DATE_PART('DOW', Year_2025) FROM DateSeries
-- DATE_PART(EXPLODE(SEQUENCE( DATE('2025-01-01'), last_day('2025-12-01'), INTERVAL 1 DAY )), 'DAYOFWEEK') as Year_2025


In [0]:
%sql
WITH DateSeries AS (
SELECT SEQUENCE (
  DATE('2025-01-01'),
  last_day('2025-12-01'),
  INTERVAL 1 DAY
) AS DateArray
)
SELECT EXPLODE(DateArray) AS Date_Col FROM DateSeries
-- SELECT * FROM variant_explode()

In [0]:
%sql
SELECT EXPLODE(sequence(1, 10, 1)) as nbrs

In [0]:
%sql
SELECT DISTINCT SDP_Event_Created_Timestamp FROM mypatronage_test where BATCH_CD = 'SCD' ORDER BY 1 desc LIMIT 1 OFFSET 1 --(OFFSET starts with 0)

In [0]:
%sql
SELECT
  Batch_CD,
  COALESCE(Monday, 0) AS Monday,
  COALESCE(Tuesday, 0) AS Tuesday,
  COALESCE(Wednesday, 0) AS Wednesday,
  COALESCE(Thursday, 0) AS Thursday,
  COALESCE(Friday, 0) AS Friday,
  COALESCE(Saturday, 0) AS Saturday,
  COALESCE(Sunday, 0) AS Sunday
FROM
  (
    SELECT
      Batch_CD,
      date_format(SDP_Event_Created_Timestamp, 'EEEE') as Weekday,
      count(*) as RecordCounts
    FROM
      mypatronage_test -- WHERE extract(year from SDP_Event_Created_Timestamp) >= extract(YEAR FROM CURRENT_DATE) --filters records based on the current year
    GROUP BY
      ALL
  ) PIVOT (
    ROUND(
      sum(RecordCounts) / weekofyear(current_date()) -1,
      0
    ) FOR Weekday IN (
      "Monday" as Monday,
      "Tuesday" as Tuesday,
      "Wednesday" as Wednesday,
      "Thursday" as Thursday,
      "Friday" as Friday,
      "Saturday" as Saturday,
      "Sunday" as Sunday
    )
  )
GROUP BY ALL 
-- WHERE Batch_CD = "CG"

In [0]:
%sql
SELECT 
   ICN, SC_Combined_Disability_Percentage as Disability_Percent, 
   PT_Indicator, filename, RecordLastUpdated, SDP_Event_Created_Timestamp as InsertedOn, 
   RecordStatus, 
   count(ICN) OVER (PARTITION BY ICN ) as cnt 
FROM mypatronage_test111
QUALIFY cnt >=2

In [0]:
%sql
SELECT ICN, PT_Indicator, SC_Combined_Disability_Percentage, filename from mypatronage where ICN in (select ICN
from pivoted_df
where `pivoted_df`.`June-24` is distinct from `pivoted_df`.`May-24` and (`pivoted_df`.`June-24` is not null and `pivoted_df`.`May-24` is not null))

In [0]:
%sql
select * from  read_files('/mnt/ci-vadir-shared/CPIDODIEX_20241005_spool.csv');


In [0]:
%sql
SELECT * FROM cloud_files_state(  "dbfs:/FileStore/streaming/scd/checkpoint/write_scd" )  -- path to checkpoint

In [0]:
%sql
SELECT DISTINCT input_file_name(), _metadata.file_modification_time,* FROM read_files(
    '/mnt/ci-vadir-shared/*.csv',
    format => 'csv', inferSchema=>True, modifiedAfter => to_date("2024-08-01 00:00:00", 'yyyy-MM-dd HH:mm:ss'))
where _metadata.file_modification_time > '2024-08-01' 

In [0]:
scd_schema1 = StructType([StructField('PTCPNT_ID', StringType()), 
                          StructField('CMBNED_DEGREE_DSBLTY', StringType()), 
                          StructField('DSBL_DTR_DT', StringType())]
)

In [0]:
new_cg_schema =  'Discharge_Revocation_Date__c STRING, Caregiver_Status__c STRING, CreatedById STRING, Dispositioned_Date__c STRING, CARMA_Case_ID__c STRING, Applicant_Type__c STRING, CreatedDate TIMESTAMP, Veteran_ICN__c STRING, Benefits_End_Date__c STRING, Caregiver_Id__c STRING, CARMA_Case_Number__c STRING, Caregiver_ICN__c STRING'


old_cg_schema = 'Discharge_Revocation_Date__c STRING, Caregiver_Status__c STRING, CreatedById STRING, Dispositioned_Date__c STRING, Applicant_Type__c STRING, CreatedDate TIMESTAMP, Veteran_ICN__c STRING, Benefits_End_Date__c STRING, Caregiver_Id__c STRING, Caregiver_ICN__c STRING'

In [0]:
%sql
select *, _metadata.file_modification_time, _metadata.file_path, rank() over(partition by Caregiver_ICN__c, Veteran_ICN__c, Applicant_Type__c order by  CreatedDate desc  ) rnk
from read_files('{cg_dir}', inferSchema=>False, header=>True,  schema=> '{use_schema}', modifiedAfter=> '{cg_max_date}') qualify rnk=1

In [0]:
cg_max_date = spark.sql("""SELECT cast(coalesce(max(from_unixtime(unix_timestamp(cast(SDP_Event_Created_Timestamp AS TIMESTAMP)))), '2023-10-30') as TIMESTAMP) cg_max_date 
            FROM mypatronage_test""").collect()[0][0]
cg_dir = '/mnt/ci-carma/landing/*.csv'
print(cg_max_date)
cg_max_date = '2023-10-30 00:00:00'

if datetime.strptime(cg_max_date, '%Y-%m-%d %H:%M:%S') < datetime.strptime('2024-08-08 00:00:00', '%Y-%m-%d %H:%M:%S'):
    use_schema = old_cg_schema
else:
    use_schema = new_cg_schema

cg_query = f"""
select *, _metadata.file_modification_time, _metadata.file_path, rank() over(partition by Caregiver_ICN__c, Veteran_ICN__c, Applicant_Type__c order by  CreatedDate desc  ) rnk
from read_files('{cg_dir}', inferSchema=>False, header=>True,  schema=> '{use_schema}', modifiedAfter=> '{cg_max_date}') qualify rnk=1"""

cg_raw_df = spark.sql(cg_query)
cg_raw_df.display()


In [0]:
%sql
SELECT
  version,
  operation,
  cast(operationMetrics.numTargetRowsInserted as INT) as Inserted,
  cast(operationMetrics.numTargetRowsUpdated as INT) as Updates  
FROM
  (describe history mypatronage_test)
where
  operation not in ("CREATE TABLE", "OPTIMIZE")
order by
  1 desc;

In [0]:
%fs 
head 'dbfs:/mnt/ci-carma/landing/caregiverevent-4ea51390-55a9-11ef-a8b3-02f025e39701.csv'

In [0]:
%sql
SHOW PARTITIONS delta.`dbfs:/user/hive/warehouse/mypatronage_test`

In [0]:
%sql
with person(name, last_name, street, city, zip) as
  (values('Bhavesh', 'Patel', '109 Austin St', 'TF', 07712) )
select name, last_name,
  struct(* EXCEPT (name, last_name)) AS address, --removes fields name and last_name
  struct(* ) AS Full_Info
from person

In [0]:
%sql
with person(name, last_name, address) as
  (values('Bhavesh', 'Patel', 
          named_struct('street', '109 Austin St', 
                        'city','TF', 
                        'zip', 07712) ))
select *
-- * except (address), address.* except(street)
  from person

In [0]:
%sql
with person(name, last_name, street, city, zip) as
  (values('Bhavesh', 'Patel', '109 Austin St', 'TF', 07712) )
select concat_ws('',*), 
      concat_ws(',',*)
from person

In [0]:

dbutils.widgets.dropdown("time_period", "Weekly", ["Weekly", "Monthly"])
time_period = dbutils.widgets.get("time_period")
print(time_period)
today = date.today()
if time_period == 'Weekly':
    start_date = today - timedelta(days=today.weekday(), weeks=1) - timedelta(days=1)
    end_date = start_date + timedelta(days=6)
else:
    first = today.replace(day=1)
    end_date = first - timedelta(days=1)
    start_date = first - timedelta(days=end_date.day)

print(start_date,  end_date)



In [0]:
%python
spark.sql("select * from {df}", df=df).display()

In [0]:
%python

def get_summary(input_df):
    """
    Get summary statistics for a dataframe.
    Input: Spark dataframe
    Ouptput: Spark dataframe with summary statistics: Total Records, Nulls, Distincts, Dups, Non-Nulls, Max, Min and % of Nulls
    """
    # Total Records
    total_records = input_df.count()

    # Null Counts
    null_counts = input_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in input_df.columns])
    null_counts_dict = null_counts.first().asDict()

    # Total Non Null Counts
    counts = input_df.select([count(col(c)).alias(c) for c in input_df.columns])
    counts_dict = counts.first().asDict()

    # Count distinct
    distinct_counts_per_column = {}
    for column in input_df.columns:
        distinct_counts = input_df.select(countDistinct(col(column)).alias(column)).first()[column]
        distinct_counts_per_column[column] = distinct_counts

    total_dups_dict = {column: counts_dict[column] - distinct_counts_per_column[column] for column in input_df.columns}

    # max for each column
    max_value = {col: input_df.agg(max(col)).collect()[0][0] for col in input_df.columns}

    # min for each column
    min_value = {col: input_df.agg(min(col)).collect()[0][0] for col in input_df.columns}

    # % of nulls
    null_percent = {column: "{:.2f}".format((null_counts_dict[column] / total_records) * 100) for column in input_df.columns }
    # Create a summary DataFrame
    summary_data = [{"CountType": "Total Records", **{c: total_records for c in input_df.columns}},
                    {"CountType": "Total Nulls", **null_counts_dict},
                    {"CountType": "Null Percentage", **null_percent},
                    {"CountType": "Distinct Values", **distinct_counts_per_column},
                    {"CountType": "Total Dups", **total_dups_dict},
                    {"CountType": "Total Non-Nulls", **counts_dict},
                    {"CountType": "Max Value", **max_value},
                    {"CountType": "Min Value", **min_value},
                    ]


    summary_df = spark.createDataFrame(summary_data)

    # Reorder columns
    summary_df = summary_df.select("CountType", *input_df.columns)

    return summary_df

# Example usage
# df = spark.table('test_correlations')
# display(get_summary(df))

In [0]:
def get_any_id(df, input_value):
    conditions = [df[col].isin(input_value) for col in df.columns]
    df_filtered = df.filter(reduce(lambda a, b: a | b, conditions))

    return df_filtered

# Example usage
# df = spark.table("test_correlations")
# df = df.drop("Last_updated_date") # remove all non numerical columns
# input_value = [1000669101, 24116350, 1082816891]
# result_df = get_any_id(df, input_value)
# display(result_df)

In [0]:
%python
spark.sql("select MVIPersonICN, Inst.* FROM {df} lateral view explode(InstitutionIDS) as Inst where Inst.InstitutionID = '200DOD' ",  df=json_format).display()

In [0]:
%sql
select * from  
read_files('/mnt/ci-vadir-shared/CPIDODIEX_20241005_spool.csv', format=>'csv', header=>true) 
where PTCPNT_ID in (40336746, 3453321, 45264911, 29646885, 57919878, 52213693, 43597650, 47771523, 37241976, 51983424, 7839780)

In [0]:
%sql
select * from mypatronage_test where Status_Begin_Date is null and Batch_Cd = 'SCD'

In [0]:
%sql
select * from  
read_files('/mnt/ci-vadir-shared/CPIDODIEX_20241005_spool.csv', format=>'csv', header=>true) 
where PTCPNT_ID in (SELECT participant_id
FROM delta.`/mnt/Patronage/SCD_Staging`
WHERE Status_Begin_Date IS NULL AND SDP_Event_Created_Timestamp = "2024-10-05T00:00:00.000+00:00" )

In [0]:
%sql
SELECT
  version,
  operation,
  cast(operationMetrics.numTargetRowsInserted as INT) as Inserted,
  cast(operationMetrics.numTargetRowsUpdated as INT) as Updates  
FROM
  (describe history mypatronage_test)
where
  operation not in ("CREATE TABLE", "OPTIMIZE")
order by
  1 desc;

In [0]:
file_list_schema = StructType(
    [
        StructField("path", StringType()),
        StructField("name", StringType()),
        StructField("size", StringType()),
        StructField("modificationTime", StringType()),
    ]
)
def all_file_list(path, cg_unix_start_time):
    """
    Creates a List of files that are at any depth inside a directory
    Parameter: Path of a directory
    Returns: Flat List with file names and it's metadata
    """
    items = dbutils.fs.ls(path)
    files = []

    for item in items:
        if item.isDir():
            # If it's a directory, recursively list its contents
            files += all_file_list(item.path, cg_unix_start_time)
        elif (
            (
                item.name.startswith("caregiverevent")
                and item.name.endswith(".csv")
                and item.modificationTime > cg_unix_start_time
            )
            or (
                (item.name.startswith("CPIDODIEX_") and item.name.endswith(".csv"))
                and "NEW" not in item.name
            )
            or (item.name.startswith("WRTS") and item.name.endswith(".txt"))
            or (item.path.startswith("dbfs:/mnt/vac20sdpasa201vba/ci-vba-edw-2/") and item.name.endswith(".parquet"))
        ):
            files.append((item.path, item.name, item.size, item.modificationTime))

    return files

def collect_data_source():
    """
    This is a helper function that will create a pyspark dataframe with all the files required to process
    Returns: Pyspark dataframe with files that are ready to process from all the
    """
    cg_start_time = datetime(
        2024, 12, 18, 23, 59, 59
    )  # this is the new datetime(2024, 12, 18, 23, 59, 59) for Caregivers data
    cg_unix_start_time = int(time.mktime(cg_start_time.timetuple())) * 1000

    others_start_time = datetime(2023, 11, 1, 0, 0, 0) # Start datetime for SCD and PAI data
    others_unix_start_time = int(time.mktime(others_start_time.timetuple())) * 1000

    # print(others_unix_start_time)

    raw_file_folders = [
        "/mnt/ci-carma/landing/",
        "/mnt/ci-vadir-shared/",
        "/mnt/ci-patronage/pai_landing/",
        "/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/",
    ]
    master_file_list = []

    for files in raw_file_folders:
        master_file_list.append(all_file_list(files, cg_unix_start_time))
    flattened_file_list = [item for sublist in master_file_list for item in sublist]
    # print(flattened_file_list)

    file_list_df = spark.createDataFrame(flattened_file_list, file_list_schema)
    # display(file_list_df)

    filtered_file_list_df = (
        file_list_df.filter(file_list_df["modificationTime"] > others_unix_start_time)
        .withColumn("dateTime", to_timestamp(col("modificationTime") / 1000))
        .orderBy(col("modificationTime"))
    )

    no_of_files = spark.sql(
        "SELECT COALESCE(COUNT(DISTINCT filename), 0) AS count from mypatronage_test"
    ).collect()[0][0]
    # print(f"Total files processed till last run: {no_of_files}")

    # display(filtered_file_list_df)
    query = f"""SELECT COALESCE(MAX(SDP_Event_Created_Timestamp), TIMESTAMP('{others_start_time}')) AS max_date from mypatronage_test """
    # print(query)
    if no_of_files == 1:
        max_processed_date = others_start_time
    else:
        max_processed_date = spark.sql(query).collect()[0][0]

    files_to_process_now = filtered_file_list_df.filter(
        filtered_file_list_df["dateTime"] > max_processed_date
    ).orderBy(col("modificationTime"))

    now = datetime.now()
    today_start_time = datetime(now.year, now.month, now.day)
    yesterday_end_time = today_start_time - timedelta(
        hours=4
    )  # To match the time of file creation date in blob storage
    yesterday_end_time_ts = int(yesterday_end_time.timestamp() * 1000)

    # files_to_process_now = files_to_process_now.filter(
    #     files_to_process_now["modificationTime"] <= yesterday_end_time_ts
    # )
    print(
        f"Upserting all files landed between date: {max_processed_date} and date: {yesterday_end_time} "
    )
    # display(files_to_process_now)
    if files_to_process_now.count() != 0:
        return files_to_process_now
    else:
        # dbutils.notebook.exit("Notebook exited because there are no files to upsert.")
        pass


In [0]:
display(dbutils.fs.ls("/mnt/vac20sdpasa201vba/"))

In [0]:
files_to_process = collect_data_source()
display(files_to_process)

In [0]:
from ftplib import FTP
ip_add = '140.20.107.4'
username = 'patelbn'
password = 'w&zuyHqy_XX5zXe4'

In [0]:
ftp = FTP(ip_add)
ftp.login(username, password)

In [0]:
%sql
SELECT DISTINCT data_date
FROM DELTA.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/`
ORDER BY DATA_DATE DESC
-- 2024-10-08 00:00:00.0000000

In [0]:
%sql
SELECT count(distinct PT35_PRMLGN_DT)
FROM DELTA.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/`
-- ORDER BY PT35_PRMLGN_DT DESC
-- LIMIT 1

In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/identity_correlations`
WHERE participant_id = 63371531

In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/duplicate_identity_correlations`
-- WHERE TreatingFacilityPersonIdentifier = 63371531
WHERE MVIPersonICN = 1008568848

In [0]:
%sql
CREATE TABLE Employee (EmpNo INT, ProjId INT, Dept INT);
INSERT INTO Employee VALUES (1,1,10);
INSERT INTO Employee VALUES (1,2,10);
INSERT INTO Employee VALUES (2,1,20);
INSERT INTO Employee VALUES (2,2,20);
INSERT INTO Employee VALUES (3,1,30);
SELECT EmpNo, COLLECT_LIST(ProjId) AS ProjId, Dept FROM Employee GROUP BY EmpNo, Dept;

In [0]:
%sql
SELECT EmpNo, collect_list(struct(Dept, ProjId)) AS ProjId FROM Employee GROUP BY EmpNo, Dept; 

In [0]:
%sql
SELECT PT_Indicator, count(1)
FROM mypatronage_test
WHERE Batch_CD ='SCD' and RecordStatus IS TRUE
GROUP BY 1

In [0]:
%sql
CREATE TEMP VIEW temp_pat as SELECT * FROM delta.`/mnt/Patronage/SCD_Staging` VERSION AS OF 128

In [0]:
%sql
SELECT d.PTCPNT_VET_ID as d_Participant_ID, d.PT_35_FLAG as d_PT, p.participant_id as pat_id, p.PT_Indicator pat_PT  FROM DELTA.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/` d
JOIN temp_pat p ON d.PTCPNT_VET_ID = p.participant_id
WHERE d.PT_35_FLAG != p.PT_Indicator

In [0]:
%sql
SELECT count(*)
-- d.PTCPNT_VET_ID as d_Participant_ID, d.PT_35_FLAG as d_PT, p.participant_id as pat_id, p.PT_Indicator pat_PT 
FROM DELTA.`/mnt/vac20sdpasa201vba/ci-vba-edw-2/DeltaTables/DW_ADHOC_RECURR.DOD_PATRONAGE_SCD_PT/` d
JOIN temp_pat p ON d.PTCPNT_VET_ID = p.participant_id
WHERE d.PT_35_FLAG != p.PT_Indicator

In [0]:
%sql
SELECT *
FROM mypatronage_test
WHERE ICN NOT IN (SELECT ICN FROM delta.`/mnt/Patronage/SCD_Staging`)
AND BATCH_CD = 'SCD'

In [0]:
%sql
SELECT * FROM delta.`/mnt/Patronage/SCD_Staging`
WHERE ICN NOT IN (SELECT DISTINCT ICN 
FROM mypatronage_test WHERE
BATCH_CD = 'SCD')

In [0]:
%sql
SELECT * FROM delta.`/mnt/Patronage/SCD_Staging`
WHERE ICN = 1079152916

In [0]:
icn_relationship = (spark.read.format("delta")
                    .load("/mnt/Patronage/identity_correlations")
                    .withColumnRenamed('MVIPersonICN', 'ICN')).persist()

In [0]:
%sql
SELECT * from delta.`/mnt/Patronage/duplicate_identity_correlations`
WHERE TreatingFacilityPersonIdentifier = 64246516

In [0]:
%sql
SELECT * from delta.`/mnt/Patronage/identity_correlations`
WHERE participant_id = 64246516

In [0]:
%sql
SELECT
  version,
  operation,
  cast(operationMetrics.numTargetRowsInserted as INT) as Inserted,
  cast(operationMetrics.numTargetRowsUpdated as INT) as Updates  
FROM
  (describe history delta.`/mnt/Patronage/SCD_Staging`)
where
  operation not in ("CREATE TABLE", "OPTIMIZE")
order by
  1 desc;

In [0]:
%sql
SELECT distinct EXPLODE(collect_list(SC_Combined_Disability_Percentage))
FROM mypatronage_test WHERE BATCH_CD = 'SCD'

In [0]:
%sql
SELECT *
FROM (
  SELECT SC_Combined_Disability_Percentage, count(*) over() AS TotRec
  FROM test_new_patronage WHERE BATCH_CD = 'SCD'
)
PIVOT (
  count(SC_Combined_Disability_Percentage)
  FOR SC_Combined_Disability_Percentage IN ( '010', '020' )
)

In [0]:
%sql
SELECT SC_Combined_Disability_Percentage, count(*) over() AS TotRec
  FROM mypatronage_test WHERE BATCH_CD = 'SCD'
  

In [0]:
from ftplib import FTP
ip_add = '10.237.64.5' #('140.20.107.4' This was provided by DoD)
username = 'patelbn'
password = 'w&zuyHqy_XX5zXe4'
ftp_directory = '/ftp/data02/pftbat/incoming' 


In [0]:
ftp = FTP()
ftp.connect(host=ip_add, port=22)
ftp.login(user=username, passwd=password)
ftp.set_pasv(True)

In [0]:
import socket

# Get the hostname of the system
hostname = socket.gethostname()

# Retrieve the IP address associated with the hostname
ip_address = socket.gethostbyname(hostname)

print(f"Your Computer Name is: {hostname}")
print(f"Your Computer IP Address is: {ip_address}")

In [0]:
import requests

# Fetch public IP address from an external service
response = requests.get('https://api.ipify.org?format=text')
public_ip = response.text

print(f"Your Public IP Address is: {public_ip}")

In [0]:
%sql
  select EDIPI,
          Batch_CD,
          SC_Combined_Disability_Percentage,
          status_begin_date,
          PT_Indicator,
          Individual_Unemployability,
          Status_Last_Update,
          Status_Termination_Date
          from mypatronage_test111
        where Batch_CD = 'CG'
        and SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and EDIPI is not null
        and Applicant_Type = 'Primary Caregiver'
        and (Status_Termination_Date is NULL OR Status_Termination_Date >= curdate()
            OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))
        and EDIPI is not null
        and sentToDoD is false

In [0]:
%sql
select count(*)
            -- edipi,
            -- Batch_CD,
            -- SC_Combined_Disability_Percentage,
            -- Status_Begin_Date,
            -- PT_Indicator,
            -- Individual_Unemployability,
            -- Status_Last_Update,
            -- Status_Termination_Date
        from mypatronage_test111
        where Batch_CD = 'SCD'
        and SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and edipi is not null
        and sentToDoD is false

In [0]:
20196 + 155

In [0]:
%sql
  select rpad(coalesce(EDIPI,''),10,' ' ) ||
            rpad(coalesce(Batch_CD,''), 3,' ') ||
            rpad(coalesce(SC_Combined_Disability_Percentage,''),3,' ') ||
            rpad(coalesce(date_format(status_begin_date, 'yyyyMMdd'),''),8,' ') ||
            rpad(coalesce(PT_Indicator,''),1,' ') ||
            rpad(coalesce(Individual_Unemployability,''),1,' ') ||  
            rpad(coalesce(date_format(Status_Last_Update, 'yyyyMMdd'),''),8,' ') ||
            rpad(coalesce(date_format(Status_Termination_Date, 'yyyyMMdd'),''),8,' ') as CG
        from mypatronage_test111
        where Batch_CD = 'CG'
        and SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and EDIPI is not null
        and Applicant_Type = 'Primary Caregiver'
        and (Status_Termination_Date is NULL OR Status_Termination_Date >= curdate()
            OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))
        and EDIPI is not null

In [0]:
%sql
  select rpad(coalesce(EDIPI,''),10,' ' ) ||
            rpad(coalesce(Batch_CD,''), 3,' ') ||
            rpad(coalesce(SC_Combined_Disability_Percentage,''),3,' ') ||
            rpad(coalesce(date_format(status_begin_date, 'yyyyMMdd'),''),8,' ') ||
            rpad(coalesce(PT_Indicator,''),1,' ') ||
            rpad(coalesce(Individual_Unemployability,''),1,' ') ||  
            rpad(coalesce(date_format(Status_Last_Update, 'yyyyMMdd'),''),8,' ') ||
            rpad(coalesce(date_format(Status_Termination_Date, 'yyyyMMdd'),''),8,' ') as CG
        from mypatronage_test111
        where Batch_CD = 'CG'
        and SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and EDIPI is not null
        and Applicant_Type = 'Primary Caregiver'
        and (Status_Termination_Date is NULL OR Status_Termination_Date >= curdate()
            OR Caregiver_Status IN ('Approved', 'Pending Revocation/Discharge'))
        and EDIPI is not null
        
        union all
        
        select rpad(coalesce(edipi,''),10,' ' ) ||
            rpad(coalesce(Batch_CD,''), 3,' ') ||
            rpad(coalesce(SC_Combined_Disability_Percentage,''),3,' ') ||
            rpad(coalesce(Status_Begin_Date,''),8,' ') ||
            rpad(coalesce(PT_Indicator,''),1,' ') ||
            rpad(coalesce(Individual_Unemployability,''),1,' ') ||
            rpad(coalesce(Status_Last_Update,''),8,' ') ||
            rpad(coalesce(Status_Termination_Date,''),8,' ') as CG
        from scd_data
        where SDP_Event_Created_Timestamp >= DATE('2025-05-16')
        and SDP_Event_Created_Timestamp <= DATE('2025-05-20')
        and edipi is not null

In [0]:
%sql
SELECT ICN, Status_Begin_Date, Status_Last_Update, RecordChangeStatus, RecordStatus, change_log
FROM mypatronage_test111
WHERE BATCH_CD = 'SCD'
AND Status_Begin_Date != Status_Last_Update
ORDER BY ICN, SDP_Event_Created_Timestamp DESC


In [0]:
%sql
SELECT *
FROM delta.`/mnt/Patronage/SCD_Staging`
WHERE ICN = 1000665065

In [0]:
display(dbutils.fs.mounts())

In [0]:
display(dbutils.fs.ls('dbfs:/mnt/ci-published/Database=MPD/'))

In [0]:
display(dbutils.fs.ls('dbfs:/mnt/ci-published/Database=MPD/lu_medalaward/'))

In [0]:
%sql
SELECT *
FROM delta.`dbfs:/mnt/ci-published/Database=MPD/lu_medalaward/`

In [0]:
df = spark.createDataFrame(dbutils.fs.ls('dbfs:/mnt/ci-published/Database=MPD/personmedal/'))


In [0]:
df.selectExpr(
    "path",
    "name",
    "size",
    "from_unixtime(modificationTime/1000) as modificationTime"
).display()

In [0]:
%sql
SELECT * FROM delta.`dbfs:/mnt/ci-published/Database=MPD/personmedal/`